In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
%matplotlib inline

from PIL import Image

from ipywidgets import interact, fixed
import ipywidgets as widgets

In [2]:
# See https://en.wikipedia.org/wiki/Gaussian_beam

def beam_waist(z, w0, wavelength_um):
    zR = np.pi * w0**2 / wavelength_um
    w_z = w0 * np.sqrt(1 + (z / zR)**2)
    # print(zR, w_z)
    return w_z
    
def gaussian_beam(x, z, w0, wavelength_um):
    w_z = beam_waist(z, w0, wavelength_um)
    temp = (w0 / w_z)**2
    temp *= np.exp(-2 * x**2 / w_z**2)
    return temp

def scaled_intensity(intensity):
    assert np.amax(intensity) <= 1.0
    assert np.amin(intensity) >= 0.0
    temp = np.copy(intensity)
    temp[temp == 0] = 1.e-6
    temp = 1.0 + np.log10(temp) / 2
    temp[temp < 0] = 0
    return temp

def absorption(z, ha):
    assert z >= 0.0
    return np.exp(-z/ha)

def make_scaled_colormap(scale_function):
    # See ImportanceOfBeingErnest's answer at https://stackoverflow.com/questions/16834861/create-own-colormap-using-matplotlib-and-plot-color-scale
    cvals  = np.linspace(0, 1)
    color_values = scaled_intensity(cvals)
    colors = [(c, c, c) for c in color_values]
    
    tuples = list(zip(cvals, colors))
    return matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)

cmap_scaled = make_scaled_colormap(scaled_intensity)

In [45]:
def plot_gaussian_beam(pixel_um=7.6, 
                       wavelength_um=0.365,
                       ha=12.0,
                       aspect_equal=True,
                       attenuate_in_resin=False,
                       cmap_name='linear',
                      ):

    min_beam_waist_um = pixel_um / 2

    aspect = 'equal' if aspect_equal else 'auto'

    gaussian_image = np.zeros((1001,101))
    center = (int(gaussian_image.shape[0]/2), int(gaussian_image.shape[1]/2))
    # print(center)

    # Create Gaussian beam image
    x = np.arange(-center[1], center[1] + 1)
    for z_index in range(0, gaussian_image.shape[0]):
        z = z_index - center[0]
        # print(z_index, z)
        gaussian_image[z+center[0], :] = gaussian_beam(x, z, min_beam_waist_um, wavelength_um)

    if attenuate_in_resin:
        # Gaussian beam attenuated by resin absorption
        attenuated_gaussian = np.copy(gaussian_image)
        for z_index in range(center[0], attenuated_gaussian.shape[0]):
            z = z_index - center[0]
            attenuated_gaussian[z_index, :] *= absorption(z, ha)
    else:
        attenuated_gaussian = np.copy(gaussian_image)

    # Show features of interest
    # Indicate start of resin region
    attenuated_gaussian[center[0], :] = 1.0
    # Indicate ha depth into resin region
    attenuated_gaussian[int(center[0] + ha), :] = 0.5
    
    if cmap_name == 'linear':
        cmap = 'gray'
    elif cmap_name == 'scaled':
        cmap = cmap_scaled
    else:
        raise ValueError(f"Color map {cmap_name} is undefined")

    fig, ax = plt.subplots(figsize=(12,12))
    im = ax.imshow(attenuated_gaussian, cmap=cmap, vmin=0, vmax=1, aspect=aspect, interpolation=None, 
              origin='lower', extent=[-center[1], center[1], -center[0], center[0]]);
    ax.set_xlabel("x (microns)")
    ax.set_ylabel("z (microns)")
    
    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel('Normalized Intensity', rotation=-90, va="bottom")


# Gaussian Beam Visualization

     pixel_um = 7.6 
     wavelength_um = 0.365
     ha = 12.0
     min_beam_waist_um = pixel_um / 2

In [46]:
# interact(plot_gaussian_beam, 
#          pixel_um=fixed(7.6), 
#          wavelength_um=fixed(0.365),
#          ha=fixed(12.0),
#          aspect_equal=True,
#          attenuate_in_resin=False,
#          cmap_name=widgets.RadioButtons(
#                 options=['linear', 'scaled'],
#                 value='linear',
#                 description='Color map',
#                 disabled=False
#             )
#         );


In [61]:
def plot_gaussian_beam2(pixel_um=7.6, 
                       wavelength_um=0.365,
                       ha=12.0,
                       aspect_equal=True,
                       attenuate_in_resin=False,
                       cmap_name='linear',
                      ):

    min_beam_waist_um = pixel_um / 2

    aspect = 'equal' if aspect_equal else 'auto'

    gaussian_image = np.zeros((1001,101))
    gaussian_image.shape
    center = (int(gaussian_image.shape[0]/2), int(gaussian_image.shape[1]/2))
    # print(center)

    # Create Gaussian beam image
    x = np.arange(-center[1], center[1] + 1)
    for z_index in range(0, gaussian_image.shape[0]):
        z = z_index - center[0]
        # print(z_index, z)
        gaussian_image[z+center[0], :] = gaussian_beam(x, z, min_beam_waist_um, wavelength_um)

    intensity_center = gaussian_image[center[0], :]
    intensity_start = gaussian_image[0, :]
    intensity_50um = gaussian_image[550, :]
    intensity_ha = gaussian_image[int(ha), :]
    
    
    if attenuate_in_resin:
        # Gaussian beam attenuated by resin absorption
        attenuated_gaussian = np.copy(gaussian_image)
        for z_index in range(center[0], attenuated_gaussian.shape[0]):
            z = z_index - center[0]
            attenuated_gaussian[z_index, :] *= absorption(z, ha)
    else:
        attenuated_gaussian = np.copy(gaussian_image)

    # Show features of interest
    # Indicate start of resin region
    attenuated_gaussian[center[0], :] = 1.0
    # Indicate ha depth into resin region
    attenuated_gaussian[int(center[0] + ha), :] = 0.5
    # Indicate ha depth of field into resin region
    attenuated_gaussian[int(center[0]) + 50, :] = 1.0
    # print(int(center[0]) + 50)
    
    if cmap_name == 'linear':
        cmap = 'gray'
    elif cmap_name == 'scaled':
        cmap = cmap_scaled
    else:
        raise ValueError(f"Color map {cmap_name} is undefined")
        
    fig = plt.figure(figsize=(12,12))
    ax1 = plt.subplot2grid((4, 7), (0, 0), colspan=3, rowspan=4)
    ax2 = plt.subplot2grid((4, 7), (1, 4), colspan=4, rowspan=2)

    im = ax1.imshow(attenuated_gaussian, cmap=cmap, vmin=0, vmax=1, aspect=aspect, interpolation=None, 
              origin='lower', extent=[-center[1], center[1], -center[0], center[0]]);
    ax1.set_xlabel("x (microns)")
    ax1.set_ylabel("z (microns)")
    
    ax2.plot(x, intensity_center, label="z = 0 microns")
    # ax2.plot(x, intensity_center, label=f"z = {int(ha)} microns")
    ax2.plot(x, intensity_50um, label="z = 50 microns (DOF)")
    ax2.plot(x, intensity_start, label="z = -500 microns")
    ax2.set_ylim(0, 1)
    ax2.set_xlabel("x (microns)")
    ax2.set_ylabel("Normalized Intensity")
    ax2.legend()
    ax2.set_title("Intensity Cross-Section")
    
    # Create colorbar
    cbar = ax1.figure.colorbar(im, ax=ax1)
    cbar.ax.set_ylabel('Normalized Intensity', rotation=-90, va="bottom")


In [62]:
interact(plot_gaussian_beam2, 
         pixel_um=fixed(7.6), 
         wavelength_um=fixed(0.365),
         ha=fixed(12.0),
         aspect_equal=True,
         attenuate_in_resin=False,
         cmap_name=widgets.RadioButtons(
                options=['linear', 'scaled'],
                value='linear',
                description='Color map',
                disabled=False
            )
        );


interactive(children=(Checkbox(value=True, description='aspect_equal'), Checkbox(value=False, description='att…